In [24]:
import spike.spike_analysis.spike_collection as sc
import spike.spike_analysis.spike_recording as sr
import spike.spike_analysis.firing_rate_calculations as fr
import spike.spike_analysis.normalization as norm
import spike.spike_analysis.single_cell as single_cell
import spike.spike_analysis.spike_collection as collection
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import behavior.boris_extraction as boris
import matplotlib.pyplot as plt
import pickle
import re

In [25]:
pd.set_option('display.max_colwidth', 0)  # 0 means unlimited in newer pandas versions

# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Don’t truncate column contents
pd.set_option("display.max_colwidth", None)

# Expand the display to the full width of the screen
pd.set_option("display.width", 0)


In [26]:
spike_collection_json_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\spike_collection.json\spike_collection.json'

In [27]:
import importlib
import spike.spike_analysis.spike_collection as sc

importlib.reload(sc)

<module 'spike.spike_analysis.spike_collection' from 'c:\\Users\\thoma\\Code\\ResearchCode\\diff_fam_social_memory_ephys\\spike\\spike_analysis\\spike_collection.py'>

In [28]:
new_sp = sc.SpikeCollection.load_collection(spike_collection_json_path)

In [29]:
rec1 = new_sp.recordings[0].event_dict.keys()

In [ ]:
def table_of_events_neurons_per_rec(spike_collection):
    """
    Returns a table with # of total neurons in recording and # of each event type or event_dict keys.
    """
    rows = []
    for rec in spike_collection.recordings:
        # Get the number of neurons in the recording
        num_neurons = rec.good_neurons

        # Get the number of events for each event type
        event_counts = {event: len(rec.event_dict[event]) for event in rec.event_dict.keys()}

        # Create a row for the table
        row = {
            'recording': rec.name,
            'num_neurons': num_neurons,
            **event_counts
        }

        # Collect rows in a list instead of using append()
        rows.append(row)

    # Create the DataFrame at the end
    table = pd.DataFrame(rows, columns=[
        'recording', 'num_neurons', 'alone_rewarded', 'alone_rewarded_baseline', 'high_comp',
        'high_comp_lose', 'high_comp_lose_baseline', 'high_comp_win', 'high_comp_win_baseline',
        'lose', 'low_comp', 'low_comp_lose', 'low_comp_lose_baseline', 'low_comp_win',
        'low_comp_win_baseline', 'overall_pretone', 'win'
    ])
    
    return table

# Usage
table = table_of_events_neurons_per_rec(new_sp)
table.head(20)


,recording,num_neurons,alone_rewarded,alone_rewarded_baseline,high_comp,high_comp_lose,high_comp_lose_baseline,high_comp_win,high_comp_win_baseline,lose,low_comp,low_comp_lose,low_comp_lose_baseline,low_comp_win,low_comp_win_baseline,overall_pretone,win
0,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,22,40,40,9,1,40,8,40,7,11,6,40,5,40,160.0,13
1,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,10,40,40,9,8,40,1,40,13,11,5,40,6,40,160.0,7
2,20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec,15,40,40,14,10,40,4,40,13,6,3,40,3,40,160.0,7
3,20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.rec,20,40,40,14,4,40,10,40,7,6,3,40,3,40,160.0,13
4,20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged.rec,15,40,0,11,0,0,11,0,1,9,1,0,8,0,NaN,19
5,20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged.rec,37,40,0,11,11,0,0,0,19,9,8,0,1,0,NaN,1
6,20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged.rec,31,38,38,14,5,38,9,38,6,6,1,38,5,38,152.0,14
7,20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged.rec,27,38,38,14,9,38,5,38,14,6,5,38,1,38,152.0,6
8,20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged.rec,19,39,39,11,1,39,10,39,6,9,5,39,4,39,156.0,14
9,20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.rec,15,39,39,11,10,39,1,39,14,9,4,39,5,39,156.0,6


In [ ]:
# save table to csv and put in the same directory 
table.to_csv('spike_collection_event_counts.csv', index=False)


In [ ]:
len(new_sp.recordings)

39